In [37]:
import pandas as pd
import numpy as np
import polars as pl
from datetime import datetime

from factorlib.utils.system import get_data_dir

In [2]:
tickers = ["OPRA", "SMCI", "LMB", "MLTX", "YPF", "CABA", "WEAV", "ELF", "EDN", "ACLS", "INTT", "ETNB", "CIR", "RCL", "NVDA", "DAKT", "TCMD", "DMAC", "IMVT", "MMMB", "ENIC", "WFRD", "IPDN", "STRL", "RMBS", "MOD", "NGL", "TDW", "TAYD", "VIST", "EXTR", "SYM", "CCL", "CMT", "CBAY", "TGLS", "BELFB", "VECT", "AEHR", "CUK", "UFPT", "AUGX", "ISEE", "TAST", "COCO", "VRT", "BWMN", "ONCY", "BLDR", "ODC", "ATEC", "NVTS", "RMTI", "AVDL", "IRS", "DFH", "CVRX", "PEN", "TGS", "GRBK", "PLPC", "SKYW", "USAP", "ACVA", "RETA", "BTBT", "TROO", "POWL", "PPSI", "FTI", "DO", "SGML", "GGAL", "PCYG", "NETI", "TRHC", "ARDX", "STVN", "NFLX", "INTA", "MORF", "RXST", "HGBL", "GE", "BZH", "BBAR", "PESI", "RIG", "NU", "TK", "JBL", "ERO", "SMHI", "IRON", "EVLV", "GENI", "ELTK", "ENVX", "META", "NCLH"]

In [27]:
raw_data_dir = get_data_dir() / 'raw'
industry_codes = (
    pl.scan_csv(raw_data_dir / 'ohclv_daily.csv', try_parse_dates=True)
    .select(
        pl.col('datadate').alias('date_index'),
        pl.col('tic').alias('ticker'),
        pl.col('gind').alias('industry'),
        pl.col('gsubind').alias('subindustry')
    )
    .filter(pl.col('ticker').is_in(tickers))
    .collect(streaming=True)
)
industry_codes = industry_codes.to_pandas().set_index('date_index')
industry_codes.index = pd.to_datetime(industry_codes.index)
industry_codes = pl.from_pandas(industry_codes.reset_index())
industry_codes

date_index,ticker,industry,subindustry
datetime[ns],str,i64,i64
2023-06-06 00:00:00,"""BELFB""",452030,45203015
2023-06-02 00:00:00,"""BELFB""",452030,45203015
2023-06-01 00:00:00,"""BELFB""",452030,45203015
2023-05-31 00:00:00,"""BELFB""",452030,45203015
2023-05-26 00:00:00,"""BELFB""",452030,45203015
2023-05-25 00:00:00,"""BELFB""",452030,45203015
2023-05-24 00:00:00,"""BELFB""",452030,45203015
2023-05-22 00:00:00,"""BELFB""",452030,45203015
2023-05-19 00:00:00,"""BELFB""",452030,45203015


In [28]:
fundamentals = pl.scan_csv(raw_data_dir / 'ratios.csv').filter(pl.col('symbol').is_in(tickers)).collect(streaming=True)
fundamentals = fundamentals.rename(mapping={'symbol': 'ticker', 'date': 'date_index'})
fundamentals = fundamentals.to_pandas().set_index('date_index')
fundamentals.index = pd.to_datetime(fundamentals.index)
fundamentals = pl.from_pandas(fundamentals.reset_index())
fundamentals = fundamentals.drop(columns=['period', 'column_1'])
fundamentals

date_index,ticker,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,debtRatio,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier,receivablesTurnover,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
datetime[ns],str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-03-31 00:00:00,"""ACLS""",3.487479,2.367288,0.61386,272.430931,157.231352,224.406102,36.283558,188.122544,0.409491,0.202177,0.208259,0.187769,0.098389,0.044171,0.067962,0.063257,0.901611,1.030084,0.202177,0.042718,0.065726,0.060043,0.061673,37.957871,0.749198,1.538606,1.339789,2.480462,0.572405,3.668,0.235242,1.05543,0.987021,13.581084,0.0,0.136048,0.935183,0.749198,26.645335,-15.428125,-15.428125,0.0,6.216882,6.216882,17.176356,22.869038,135.002259,126.251859,126.251859,-1.421147,17.176356,0.0,78.237711,6.216882
2022-12-31 00:00:00,"""ACLS""",3.537009,2.425882,0.747969,232.914535,139.376094,196.807075,35.847058,160.960017,0.411654,0.21082,0.247306,0.214215,0.133808,0.056225,0.085412,0.07327,0.866192,1.173064,0.21082,0.045789,0.06956,0.063423,0.065036,38.026441,2.636791,1.519119,1.567098,2.510666,0.645735,3.814242,0.262471,3.728605,3.612619,13.16656,0.0,0.460002,0.968893,2.636791,99.580146,-32.147097,-32.147097,0.0,3.903799,3.903799,9.790729,11.426311,21.967441,21.2841,21.2841,0.268079,9.790729,0.0,36.652529,3.903799
2022-09-30 00:00:00,"""ACLS""",3.729226,2.482589,1.484451,276.954053,161.978404,230.268444,38.630477,191.637967,0.450839,0.232146,0.196396,0.175774,0.105001,0.04445,0.065513,0.076176,0.894999,0.846002,0.232146,0.0515,0.075903,0.06891,0.070548,39.911478,1.371293,1.473859,1.317908,2.329767,0.55563,3.32446,0.25288,1.938778,1.832147,10.36406,0.0,0.279267,0.945001,1.371293,54.983677,-18.182102,-18.182102,0.0,3.251226,3.251226,8.72323,12.406885,33.054119,31.236171,31.236171,-1.385435,8.72323,0.0,37.487941,3.251226
2022-06-30 00:00:00,"""ACLS""",4.443538,2.832497,1.877277,241.136285,157.182776,216.641038,36.451195,180.189843,0.448424,0.244804,0.217907,0.19979,0.08314,0.054438,0.075289,0.082192,0.91686,0.890128,0.244804,0.057806,0.079947,0.072417,0.074029,43.316,0.073567,1.383027,1.513667,2.469055,0.572582,3.400578,0.272474,0.104303,0.048314,8.676789,0.0,0.015607,0.46321,0.073567,3.135332,-1.862925,-1.862925,0.0,3.092352,3.092352,8.206028,10.268306,1135.074822,525.777706,525.777706,1.426154,8.206028,0.0,31.847685,3.092352
2022-03-31 00:00:00,"""ACLS""",4.398096,2.821264,2.014556,213.316904,161.166487,213.765175,40.133094,173.632081,0.440905,0.240296,0.225364,0.204396,0.093041,0.053201,0.074407,0.077081,0.906959,0.937862,0.240296,0.060301,0.084336,0.076194,0.077777,32.22859,0.546357,1.398594,1.711069,2.242538,0.558429,3.190294,0.260286,0.775154,0.729944,8.937915,0.0,0.126575,0.941676,0.546357,24.802695,-17.145709,-17.145709,0.0,4.489748,4.489748,12.333283,15.085037,103.473641,97.438683,97.438683,0.896722,12.333283,0.0,47.699855,4.489748
2021-12-31 00:00:00,"""ACLS""",4.124457,2.659738,1.96432,185.282521,151.02031,19

In [29]:
full_data = industry_codes.join(fundamentals, on=['ticker', 'date_index'], how='inner')
full_data

date_index,ticker,industry,subindustry,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,debtRatio,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier,receivablesTurnover,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
datetime[ns],str,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-03-31 00:00:00,"""BELFB""",452030,45203015,2.946326,1.438544,0.599429,230.772322,125.74368,182.646445,45.680233,136.966212,0.311377,0.113947,0.108713,0.084552,0.222246,0.025364,0.052405,0.044163,0.777754,0.954069,0.113947,0.209409,0.432675,0.292704,0.302005,19.97762,0.139929,2.066172,1.581645,1.970218,0.715742,2.930024,0.299976,7.859477,6.103641,36.339869,0.05689,0.097683,0.776596,0.139929,3.2134,-4.476203,-5.741814,0.05689,0.28949,0.28949,0.467068,1.381011,6.15698,4.781489,4.781489,-0.515577,0.467068,0.010299,4.646941,0.28949
2022-09-30 00:00:00,"""BELFB""",452030,45203015,2.752841,1.270159,0.517172,211.971852,117.224278,169.49131,47.202488,122.288822,0.289942,0.12412,0.116395,0.093103,0.200116,0.02988,0.070109,0.052938,0.799884,0.937763,0.12412,0.236353,0.554567,0.346214,0.356734,23.369703,0.098727,2.34635,1.721927,1.906679,0.767759,3.170797,0.320935,1.035331,0.869813,5.679779,0.049795,0.072708,0.84013,0.098727,2.188855,-6.255082,-10.404992,0.049795,1.335282,1.335282,1.773221,4.761459,29.029244,24.388339,24.388339,-0.65232,1.773221,0.002614,15.62889,1.335282
2022-06-30 00:00:00,"""BELFB""",452030,45203015,2.804676,1.278501,0.511301,211.368835,115.516065,167.634408,50.366049,117.268359,0.266468,0.064999,0.083789,0.099887,-0.192135,0.031415,0.075391,0.026805,1.192135,1.289077,0.064999,0.25226,0.605394,0.364397,0.3771,14.232349,0.138098,2.399876,1.726839,1.786918,0.779112,2.826895,0.3145,1.510795,1.390373,5.263873,0.048304,0.110768,0.920292,0.138098,2.60571,-12.545817,-27.66325,0.048304,0.861052,0.861052,1.140828,2.855285,11.191245,10.299215,10.299215,0.012194,1.140828,0.004229,17.585708,0.861052
2022-03-31 00:00:00,"""BELFB""",452030,45203015,3.052723,1.324178,0.468156,250.108215,136.272004,197.942523,56.683139,141.259384,0.249594,0.022053,0.048479,0.03704,0.235969,0.009733,0.023515,0.007339,0.764031,2.198342,0.022053,0.264959,0.640117,0.377846,0.390287,4.382267,-0.056205,2.415911,1.459368,1.587774,0.660444,2.180684,0.262777,-0.618899,-0.781852,4.092579,0.16252,-0.056671,1.263294,-0.056205,-1.096984,3.798039,6.366475,0.16252,1.037069,1.037069,1.633574,11.025818,-22.81763,-28.825369,-28.825369,-0.306805,1.633574,0.003685,42.366998,1.037069
2021-12-31 00:00:00,"""BELFB""",452030,45203015,2.937328,1.329242,0.551334,216.145348,116.284936,169.581049,55.029339,114.55171,0.266856,0.071013,0.067424,0.054423,0.192823,0.015645,0.038363,0.026133,0.807177,0.949469,0.071013,0.261891,0.642168,0.378576,0.391049,19.789773,0.047341,2.452039,1.688678,1.635491,0.773961,2.474572,0.287475,0.509024,0.088474,4.953557,0.116384,0.043128,0.17381,0.047341,0.922384,-1.210376,-1.472048,0.116384,0.772233,0.772233,1.09552

In [60]:
# the following list was determined from chatgpt as the best metrics to convert to categorical data
subindustry_metrics = [
    'grossProfitMargin',
    'operatingProfitMargin',
    'pretaxProfitMargin',
    'netProfitMargin',
    'returnOnAssets',
    'returnOnEquity',
    'returnOnCapitalEmployed',
    'debtRatio',
    'debtEquityRatio',
    'longTermDebtToCapitalization',
    'totalDebtToCapitalization',
    'currentRatio',
    'quickRatio',
    'cashRatio',
    'daysOfSalesOutstanding',
    'daysOfInventoryOutstanding',
    'operatingCycle',
    'daysOfPayablesOutstanding',
    'cashConversionCycle',
    'priceToBookRatio',
    'priceToSalesRatio',
    'priceEarningsRatio',
    'priceToFreeCashFlowsRatio',
    'priceToOperatingCashFlowsRatio',
    'priceCashFlowRatio'
]
# Then sort the dataframe by date
subindustry_fundamentals = full_data.to_pandas()
subindustry_fundamentals = subindustry_fundamentals.sort_values(by=['date_index'])

grouped_df = subindustry_fundamentals.groupby([pd.Grouper(key='date_index', freq='Q'), 'subindustry'])[subindustry_metrics].mean().reset_index()

# Merge this back to your original dataframe
merged_df = pd.merge(subindustry_fundamentals[['date_index', 'ticker', 'subindustry']], grouped_df,  how='inner', on=['date_index', 'subindustry'])
merged_df

,date_index,ticker,subindustry,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,...,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio
0,1990-12-31,HGBL,40203010,1.000000,0.195122,0.195122,0.073171,0.014320,0.109091,0.067511,...,0.000000,NaN,0.000000,NaN,6.545455,4.390244,15.000000,32.727273,7.058824,7.058824
1,1990-12-31,MOD,25101010,0.264032,0.049012,0.184980,0.115415,0.036300,0.071115,0.019764,...,88.839957,143.480273,38.184748,105.295526,1.366186,2.217217,4.802705,11.309597,7.791056,7.791056
2,1990-12-31,SKYW,20302010,0.065934,-0.036630,-0.036630,-0.021978,-0.007926,-0.017493,-0.015649,...,21.529412,35.705236,16.588235,19.117001,0.874636,1.098901,-12.500000,-25.000000,16.666667,16.666667
3,1990-12-31,GE,20105010,0.471857,0.206695,0.099723,0.073024,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,0.000000,2.170208,7.429764,38.841679,13.226261,13.226261
4,1991-09-30,SKYW,20302010,0.295732,0.042683,0.033537,0.021341,0.009537,0.020468,0.023102,...,25.714286,41.354530,19.480519,21.874010,0.582765,0.607639,7.118058,24.913204,6.429214,6.429214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3201,2023-03-31,ACVA,20201070,0.500794,-0.174159,-0.149240,-0.152141,-0.018289,-0.037845,-0.035811,...,0.000000,220.792135,584.364011,-363.571875,4.260191,17.126305,-28.142190,54.241386,47.667552,47.667552
3202,2023-03-31,CMT,15101010,0.178309,0.081150,0.078095,0.058810,0.028300,0.047681,0.052591,...,27.529230,75.014232,36.592632,38.421599,1.233897,1.521901,6.469575,60.095167,32.588728,32.588728
3203,2023-03-31,HGBL,40203010,0.598182,0.234409,0.230315,0.170299,0.037446,0.055218,0.072092,...,47.083146,100.209801,220.085393,-119.875592,2.016957,6.220490,9.131741,11.668336,11.552239,11.552239
3204,2023-03-31,PPSI,20104020,0.260139,0.006465,0.014341,0.022217,0.005271,0.010999,0.003015,...,137.116301,220.303089,92.731173,127.571916,2.001210,4.042412,45.487829,25.836813,22.550032,22.550032


In [65]:
# NEED TO RESAMPLE AND MAKE SURE ITS CORRECT
fundamental_data_dir = get_data_dir() / 'fundamental'
categorical_fundamentals = merged_df.drop(columns=['subindustry']).set_index('date_index')
categorical_fundamentals = categorical_fundamentals.groupby('ticker').resample('B').ffill().reset_index(level=0, drop=True)
categorical_fundamentals

,ticker,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,debtRatio,debtEquityRatio,...,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio
date_index,,,,,,,,,,,,,,,,,,,,,
2001-12-31,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-01,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-02,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-03,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-04,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27,YPF,0.267015,0.321531,0.182347,0.127733,0.024685,0.062312,0.078972,0.297023,0.749780,...,50.264346,86.709192,58.474169,28.235023,0.243472,0.499092,0.976828,4.942414,1.669913,1.669913
2023-03-28,YPF,0.267015,0.321531,0.182347,0.127733,0.024685,0.062312,0.078972,0.297023,0.749780,...,50.264346,86.709192,58.474169,28.235023,0.243472,0.499092,0.976828,4.942414,1.669913,1.669913
2023-03-29,YPF,0.267015,0.321531,0.182347,0.127733,0.024685,0.062312,0.078972,0.297023,0.749780,...,50.264346,86.709192,58.474169,28.235023,0.243472,0.499092,0.976828,4.942414,1.669913,1.669913


In [66]:
categorical_fundamentals.to_csv(fundamental_data_dir / 'subindustry_fundamentals.csv')
categorical_fundamentals

,ticker,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,debtRatio,debtEquityRatio,...,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio
date_index,,,,,,,,,,,,,,,,,,,,,
2001-12-31,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-01,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-02,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-03,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
2002-01-04,ACLS,0.278583,-0.726667,-0.511650,-0.308605,-0.040319,-0.046684,-0.133826,0.003956,0.004619,...,318.268684,427.943566,41.494148,386.449418,1.709572,13.793878,-10.569581,-6.374534,-32.454747,-32.454747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-27,YPF,0.267015,0.321531,0.182347,0.127733,0.024685,0.062312,0.078972,0.297023,0.749780,...,50.264346,86.709192,58.474169,28.235023,0.243472,0.499092,0.976828,4.942414,1.669913,1.669913
2023-03-28,YPF,0.267015,0.321531,0.182347,0.127733,0.024685,0.062312,0.078972,0.297023,0.749780,...,50.264346,86.709192,58.474169,28.235023,0.243472,0.499092,0.976828,4.942414,1.669913,1.669913
2023-03-29,YPF,0.267015,0.321531,0.182347,0.127733,0.024685,0.062312,0.078972,0.297023,0.749780,...,50.264346,86.709192,58.474169,28.235023,0.243472,0.499092,0.976828,4.942414,1.669913,1.669913


In [71]:
non_cat_fundamentals = fundamentals[[ticker for ticker in fundamentals.columns if ticker not in subindustry_metrics]]
non_cat_fundamentals.write_csv(fundamental_data_dir / 'non_cat_fundamentals.csv')
non_cat_fundamentals

date_index,ticker,effectiveTaxRate,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier,receivablesTurnover,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
datetime[ns],str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-03-31 00:00:00,"""ACLS""",0.098389,0.901611,1.030084,0.202177,37.957871,0.749198,1.538606,1.339789,2.480462,0.572405,3.668,0.235242,1.05543,0.987021,13.581084,0.0,0.136048,0.935183,0.749198,26.645335,-15.428125,-15.428125,0.0,6.216882,-1.421147,17.176356,0.0,78.237711,6.216882
2022-12-31 00:00:00,"""ACLS""",0.133808,0.866192,1.173064,0.21082,38.026441,2.636791,1.519119,1.567098,2.510666,0.645735,3.814242,0.262471,3.728605,3.612619,13.16656,0.0,0.460002,0.968893,2.636791,99.580146,-32.147097,-32.147097,0.0,3.903799,0.268079,9.790729,0.0,36.652529,3.903799
2022-09-30 00:00:00,"""ACLS""",0.105001,0.894999,0.846002,0.232146,39.911478,1.371293,1.473859,1.317908,2.329767,0.55563,3.32446,0.25288,1.938778,1.832147,10.36406,0.0,0.279267,0.945001,1.371293,54.983677,-18.182102,-18.182102,0.0,3.251226,-1.385435,8.72323,0.0,37.487941,3.251226
2022-06-30 00:00:00,"""ACLS""",0.08314,0.91686,0.890128,0.244804,43.316,0.073567,1.383027,1.513667,2.469055,0.572582,3.400578,0.272474,0.104303,0.048314,8.676789,0.0,0.015607,0.46321,0.073567,3.135332,-1.862925,-1.862925,0.0,3.092352,1.426154,8.206028,0.0,31.847685,3.092352
2022-03-31 00:00:00,"""ACLS""",0.093041,0.906959,0.937862,0.240296,32.22859,0.546357,1.398594,1.711069,2.242538,0.558429,3.190294,0.260286,0.775154,0.729944,8.937915,0.0,0.126575,0.941676,0.546357,24.802695,-17.145709,-17.145709,0.0,4.489748,0.896722,12.333283,0.0,47.699855,4.489748
2021-12-31 00:00:00,"""ACLS""",0.210246,0.789754,0.972856,0.226216,36.840063,0.804089,1.397583,1.969965,3.055884,0.595946,3.241568,0.273066,1.144586,1.053972,8.857877,0.0,0.185279,0.920832,0.804089,38.926456,-12.631422,-12.631422,0.0,4.606056,0.569423,12.069365,0.0,45.283993,4.606056
2021-09-30 00:00:00,"""ACLS""",0.195774,0.804226,0.940073,0.205972,28.679275,1.39012,1.3895,2.257868,2.822108,0.509024,2.916224,0.248522,1.972747,1.874079,8.083162,0.0,0.374433,0.949985,1.39012,71.834962,-19.993956,-19.993956,0.0,3.082489,0.308663,8.926421,0.0,35.495244,3.082489
2021-06-30 00:00:00,"""ACLS""",0.168894,0.831106,0.947952,0.162941,18.83595,0.645008,1.337052,1.851478,2.045952,0.432922,2.530568,0.222926,0.915254,0.884224,6.524661,0.0,0.20929,0.966097,0.645008,35.674769,-29.495694,-29.495694,0.0,2.754946,1.259993,9.242801,0.0,44.520658,2.754946
2021-03-31 00:00:00,"""ACLS""",0.094555,0.905445,0.894398,0.153266,19.776482,0.314699,1.290432,1.748617,1.886081,0.437783,2.419035,0.210993,0.447813,0.40786,6.13276,0.0,0.11371,0.910783,0.314699,18.662546,-11.208612,-11.208612,0.0,2.84081,1.849374,10.433733,0.0,56.071285,2.84081
